In [1]:
!pip install gym

In [2]:
!git clone https://github.com/ClementRomac/gym-tictactoe
%cd gym-tictactoe
!python setup.py install

Cloning into 'gym-tictactoe'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 42 (delta 3), reused 1 (delta 0), pack-reused 30
Unpacking objects: 100% (42/42), done.
/content/gym-tictactoe
running install
running bdist_egg
running egg_info
creating gym_tictactoe.egg-info
writing gym_tictactoe.egg-info/PKG-INFO
writing dependency_links to gym_tictactoe.egg-info/dependency_links.txt
writing requirements to gym_tictactoe.egg-info/requires.txt
writing top-level names to gym_tictactoe.egg-info/top_level.txt
writing manifest file 'gym_tictactoe.egg-info/SOURCES.txt'
writing manifest file 'gym_tictactoe.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/gym_tictactoe
copying gym_tictactoe/tic_tac_toe.py -> build/lib/gym_tictactoe
copying gym_tictactoe/__init__.py -> build/lib/g

In [3]:
import gym
import gym_tictactoe
import random
import numpy as np
env = gym.make('TicTacToe-v1', symbols=[-1, 1], board_size=3, win_size=3) 

In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [5]:
def build_model():
    model = Sequential()
    model.add(Dense(128, input_dim=9, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    model.compile(loss='mse', optimizer=Adam())
    return model

In [6]:
us0_model = build_model()

us0_weight_dir = '/content/drive/MyDrive/Colab Notebooks/Reinforcement Learning/us0_weight/us0.ckpt'
us0_model.load_weights(us0_weight_dir)

In [7]:
us1_model = build_model()

us1_weight_dir = '/content/drive/MyDrive/Colab Notebooks/Reinforcement Learning/us1_weight/us1.ckpt'
us1_model.load_weights(us1_weight_dir)

In [8]:
def us0_predictor(s):
    return np.argmax(us0_model.predict(np.array(s).reshape(-1, 9))[0])

def us1_predictor(s):
    return np.argmax(us1_model.predict(np.array(s).reshape(-1, 9))[0])

In [9]:
def extra_us0_predictor(s, n):
    lst = us0_model.predict(np.array(s).reshape(-1, 9))[0]
    for i in range(n-1):
        iMax = np.argmax(lst)
        lst[iMax] = -1
    return np.argmax(lst)
def extra_us1_predictor(s, n):
    lst = us1_model.predict(np.array(s).reshape(-1, 9))[0]
    for i in range(n-1):
        iMax = np.argmax(lst)
        lst[iMax] = -1
    return np.argmax(lst)

In [10]:
def us0_play_with_AI():
    user = 0
    done = False
    reward = 0
    render = True
    # Reset the env before playing
    state = env.reset()

    while not done:
        env.render(mode=None)
        if user == 0:
            usr_action = int(input("입력 : "))
            while 0 > usr_action or usr_action > 8 or state[usr_action] != 0:
                print("잘못 입력하셨습니다.")
                usr_action = int(input("입력 : "))
            state, reward, done, infos = env.step(usr_action, -1)
        elif user == 1:
            nTry = 2
            us1_action = us1_predictor(state)
            while state[us1_action] != 0:
                us1_action = extra_us1_predictor(state, nTry)
                nTry += 1
            state, reward, done, infos = env.step(us1_action, 1)
          
        # If the game isn't over, change the current player
        if not done:
            user = 0 if user == 1 else 1
        else :
            if reward == 10:
                print("Draw !")
            elif reward == -20:
                print("Infos : " + str(infos))
                if user == 0:
                    print("AI wins!")
                elif user == 1:
                    print("User wins!")
            elif reward == 20:
                if user == 0:
                    print("User wins!")
                elif user == 1:
                    print("AI wins!")
    env.render(mode=None)

In [14]:
us0_play_with_AI()

 -------------
 |   |   |   |
 -------------
 |   |   |   |
 -------------
 |   |   |   |
 -------------

입력 : 4
 -------------
 |   |   |   |
 -------------
 |   | x |   |
 -------------
 |   |   |   |
 -------------

 -------------
 |   |   | o |
 -------------
 |   | x |   |
 -------------
 |   |   |   |
 -------------

입력 : 0
 -------------
 | x |   | o |
 -------------
 |   | x |   |
 -------------
 |   |   |   |
 -------------

 -------------
 | x |   | o |
 -------------
 |   | x |   |
 -------------
 |   |   | o |
 -------------

입력 : 5
 -------------
 | x |   | o |
 -------------
 |   | x | x |
 -------------
 |   |   | o |
 -------------

 -------------
 | x |   | o |
 -------------
 |   | x | x |
 -------------
 |   | o | o |
 -------------

입력 : 1
 -------------
 | x | x | o |
 -------------
 |   | x | x |
 -------------
 |   | o | o |
 -------------

AI wins!
 -------------
 | x | x | o |
 -------------
 |   | x | x |
 -------------
 | o | o | o |
 -------------



In [ ]:
def us1_play_with_AI():
    user = 0
    done = False
    reward = 0
    render = True
    # Reset the env before playing
    state = env.reset()

    while not done:
        env.render(mode=None)
        if user == 0:
            nTry = 2
            us0_action = us0_predictor(state)
            while state[us0_action] != 0:
                us0_action = extra_us0_predictor(state, nTry)
                nTry += 1
            state, reward, done, infos = env.step(us0_action, -1)
        elif user == 1:
            usr_action = int(input("입력 : "))
            while 0 > usr_action or usr_action > 8 or state[usr_action] != 0:
                print("잘못 입력하셨습니다.")
                usr_action = int(input("입력 : "))
            state, reward, done, infos = env.step(usr_action, 1)

        # If the game isn't over, change the current player
        if not done:
            user = 0 if user == 1 else 1
        else :
            if reward == 10:
                print("Draw !")
            elif reward == -20:
                print("Infos : " + str(infos))
                if user == 0:
                    print("User wins!")
                elif user == 1:
                    print("AI wins!")
            elif reward == 20:
                if user == 0:
                    print("AI wins!")
                elif user == 1:
                    print("User wins!")
    env.render(mode=None)

In [ ]:
us1_play_with_AI()

 -------------
 |   |   |   |
 -------------
 |   |   |   |
 -------------
 |   |   |   |
 -------------

 -------------
 |   |   |   |
 -------------
 |   | x |   |
 -------------
 |   |   |   |
 -------------

입력 : 0
 -------------
 | o |   |   |
 -------------
 |   | x |   |
 -------------
 |   |   |   |
 -------------

 -------------
 | o |   |   |
 -------------
 |   | x |   |
 -------------
 |   | x |   |
 -------------

입력 : 2
 -------------
 | o |   | o |
 -------------
 |   | x |   |
 -------------
 |   | x |   |
 -------------

 -------------
 | o |   | o |
 -------------
 | x | x |   |
 -------------
 |   | x |   |
 -------------

입력 : 6
 -------------
 | o |   | o |
 -------------
 | x | x |   |
 -------------
 | o | x |   |
 -------------

AI wins!
 -------------
 | o | x | o |
 -------------
 | x | x |   |
 -------------
 | o | x |   |
 -------------

